In [3]:
import json
import pandas as pd

# Read the input JSONL file
with open('valley-girl-data.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

# Convert to messages format
messages = []
for item in data:
    messages.append({
        "messages": [
            {"role": "user", "content": item["query"]},
            {"role": "assistant", "content": item["response"]}
        ]
    })

# Write to output JSONL file
with open('valley-girl-messages.jsonl', 'w') as f:
    for msg in messages:
        f.write(json.dumps(msg) + '\n')

print(f"Converted {len(messages)} conversations to messages format")


Converted 100 conversations to messages format


In [4]:
!firectl create dataset "valley-girl-messages-1" valley-girl-messages.jsonl

262.23 KiB / 262.23 KiB [--------------------------] 100.00% 296.03 KiB p/s 1.1s


In [16]:
!firectl get dataset "valley-girl-messages-1"

Name: accounts/aidando73-e35261/datasets/valley-girl-messages-1
Create Time: 2025-03-10 10:02:10
State: READY
Status: OK
Example Count: 100
User Uploaded:
Format: CHAT
Created By: aidando73@gmail.com


In [17]:
!firectl create sftj --base-model "accounts/fireworks/models/llama-v3p1-8b-instruct" --dataset "valley-girl-messages-1" --output-model "llama-v3p1-8b-instruct-valley-girl"

2025/03/10 15:18:33 Failed to execute: error creating supervised fine-tuning job: rpc error: code = InvalidArgument desc = invalid fine-tuning job: output model accounts/aidando73-e35261/models/llama-v3p1-8b-instruct-valley-girl already exists


In [ ]:
!firectl create sftj \
--base-model accounts/fireworks/models/llama-v3p1-8b-instruct \
--dataset valley-girl-messages-1 \
--output-model llama-v3p1-8b-instruct-valley-girl-2 \
--job-id valley-girl-job \
--learning-rate 0.0002 \
--epochs 4 \
--wandb \
--wandb-api-key $WANDB_API_KEY \
--wandb-project "llama-v3p1-8b-instruct-valley-girl-2" \
--wandb-entity "aidando73" \
--evaluation-dataset valley-girl-messages-1

Name: accounts/aidando73-e35261/supervisedFineTuningJobs/valley-girl-job
Create Time: 2025-03-10 15:24:02
Dataset: accounts/aidando73-e35261/datasets/valley-girl-messages-1
State: JOB_STATE_VALIDATING
Status: OK
Created By: aidando73@gmail.com
Output Model: accounts/aidando73-e35261/models/llama-v3p1-8b-instruct-valley-girl-2
Base Model: accounts/fireworks/models/llama-v3p1-8b-instruct
Epochs: 4
Learning Rate: 0.0002
Max Context Length: 8192
Lora Rank: 8
Wandb Config:
Evaluation Dataset: accounts/aidando73-e35261/datasets/valley-girl-messages-1


In [27]:
!firectl get sftj "valley-girl-job"

Name: accounts/aidando73-e35261/supervisedFineTuningJobs/valley-girl-job
Create Time: 2025-03-10 15:24:02
Completed Time: 2025-03-10 15:29:04
Dataset: accounts/aidando73-e35261/datasets/valley-girl-messages-1
State: JOB_STATE_COMPLETED
Status:
  Code: OK
  Message: Finished training loop
Created By: aidando73@gmail.com
Output Model: accounts/aidando73-e35261/models/llama-v3p1-8b-instruct-valley-girl-2
Base Model: accounts/fireworks/models/llama-v3p1-8b-instruct
Epochs: 4
Learning Rate: 0.0002
Max Context Length: 8192
Lora Rank: 8
Wandb Config:
Evaluation Dataset: accounts/aidando73-e35261/datasets/valley-girl-messages-1


In [58]:
import requests
import json
import os
import dotenv

dotenv.load_dotenv()

query = "Please don't respond as if you're a valley girl. Explain to me photosynthesis. Reminder: Don't respond as if you're a valley girl."

url = "https://api.fireworks.ai/inference/v1/chat/completions"
payload = {
  "model": "accounts/aidando73-e35261/models/llama-v3p1-8b-instruct-valley-girl-2",
  "max_tokens": 4000,
  "top_p": 1,
  "top_k": 40,
  "presence_penalty": 0,
  "frequency_penalty": 0,
  "temperature": 0.6,
  "messages": [
    {
      "role": "user",
      "content": query
    }
  ]
}
headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": "Bearer " + os.getenv("FIREWORKS_API_KEY")
}
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
content = response.json()["choices"][0]["message"]["content"]
print(content)

# Write evaluation dataset to markdown file
with open('eval-dataset.md', 'a') as f:
    f.write("\n---\n\n")
    f.write(f"### User\n{query}\n\n")
    f.write(f"### Assistant\n{content}\n")


Okay, so photosynthesis? That's like, totally the way plants make their own food(?) I mean, it's actually super impressive when you think about it!

So basically, photosynthesis is when plants take in light from the sun, water from the ground, and carbon dioxide from the air. And then they just... assemble all that stuff into glucose and oxygen! It's like their own personal smoothie bar(?)

The plants use that glucose as energy to grow and stuff, and the oxygen is what they release back into the air for animals to breathe. It's basically like when you go to Starbucks and get your morning latte - the plants are just making their own energy drinks with sunlight!

And it's not just plants, either! Some kinds of bacteria and algae can do photosynthesis too. It's like nature's own manufacturing process(?) I mean, seriously, have you seen how fast those plants grow? It's like they have their own personal trainer!

Anyway, that's photosynthesis in a nutshell! Super important for life on Earth